In [ ]:
%load_ext autoreload
%autoreload 2

from base64 import b64encode
import copy
from datetime import datetime
import getopt
import io
from io import BytesIO
import os
import sys

# standard numeric/scientific libraries
import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal as sps
import scipy.fftpack as fftpack 

# plotting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('figure', figsize=(20, 12))

# image display
from PIL import Image
import IPython.display 
from IPython.display import HTML

module_path = os.path.abspath(os.path.join('../lddecode'))
if module_path not in sys.path:
    sys.path.append(module_path)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lddecode.utils import *
import lddecode.core as core

# Notebook-only functions go here

# Draws a uint16 image, downscaled to uint8
def draw_raw_bwimage(bm, x = 2800, y = 525, hscale = 1, vscale = 2, outsize = None):
    if y is None:
        y = len(bm) // x
        
    if outsize is None:
        outsize = (x * hscale, y * vscale)
    
    bmf = np.uint8(bm[0:x*y] / 256.0)
    print(bmf.shape)
    if x is not None:
        bms = (bmf.reshape(len(bmf)//x, -1))
    else:
        bms = bmf
    
    print(bms.dtype, bms.shape, bms[:][0:y].shape)
    im = Image.fromarray(bms[0:y])
    im = im.resize(outsize)
    b = BytesIO()
    im.save(b, format='png')
    return IPython.display.Image(b.getvalue())


In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

matplotlib.rc('figure', figsize=(20, 12))

In [ ]:
#filename = '/home/cpage/lddecode.comb/superdon8071.lds'
filename = '/home/cpage/ld-decode-testdata/ve-snw-cut.lds'
outname = 'devbook'
system = 'NTSC'
foutput = False
loader = make_loader(filename)

firstframe = 0
req_frames = 3

ldd = core.LDdecode(filename, outname, loader, system=system)
ldd.roughseek(firstframe * 2)
ldd.blackIRE = 7.5

fields = []
for i in range(0, req_frames * 2):
    fields.append(ldd.readfield())
    

In [ ]:
def drawImage(rgb, x=640, y=480):
    im = Image.fromarray(rgb)
    imscaled = im.resize((x,y))

    b = BytesIO()
    imscaled.save(b, format='png')
    return IPython.display.Image(b.getvalue())


In [ ]:
class CombNTSC:
    ''' *partial* NTSC comb filter class - only enough to do VITS calculations ATM '''
    
    def __init__(self, field):
        self.field = field
        self.cbuffer1D = self.split1D()
        self.cbuffer2D = self.split2D()

    def getlinephase(self, line):
        ''' determine if a line has positive color burst phase '''
        fieldID = self.field.fieldPhaseID
        
        fieldPositivePhase = (fieldID == 1) | (fieldID == 4)
        
        return fieldPositivePhase if ((line % 2) == 0) else not fieldPositivePhase

    def split1D(self, subset = None):
        ''' 
        prev_field: Compute values for previous field
        subset: a slice computed by lineslice_tbc (default: whole field) 
        
        NOTE:  first and last two returned values will be zero, so slice accordingly
        '''
        
        data = self.field.dspicture
        
        if subset:
            data = data[subset]
            
        # this is a translation of this code from tools/ld-chroma-decoder/comb.cpp:
        #
        # for (qint32 h = configuration.activeVideoStart; h < configuration.activeVideoEnd; h++) {
        #  qreal tc1 = (((line[h + 2] + line[h - 2]) / 2) - line[h]);
                        
        fldata = data.astype(np.float32)
        cbuffer = np.zeros_like(fldata)
        
        cbuffer[2:-2] = (fldata[:-4] + fldata[4:]) / 2
        cbuffer[2:-2] -= fldata[2:-2]
        
        return cbuffer / 2

    def lsl(self, line):
        return slice(910 * line, 910 * (line + 1))
    
    def split2D(self, first = 10, last = 262):
        adaptive_level = 30
        kfilter = [1.0] # sps.firwin(5, .1)
        #kfilter = [-.2, 1.0]
        #kfilter = sps.firwin(3, .25)

        blacklevel = fields[0].hz_to_output(ldd.rf.iretohz(0))
        blackline = np.full(910, blacklevel)

        cbuf1D = self.cbuffer1D
        cbuf = np.zeros_like(cbuf1D)

        def procline(l, gap):
            curline = self.field.dspicture[self.lsl(l)]

            if inrange((l + gap), first, last):
                dline = self.field.dspicture[self.lsl(l + gap)].astype(np.float32)
                dcbuf = self.cbuffer1D[self.lsl(l + gap)]

                k = np.fabs(curline - dline) #+ np.fabs(np.roll(curline, 1) - np.roll(nextline, 1))
                kf = sps.lfilter(kfilter, [1.0], k)
                k_out = np.clip(1 - (kf / fields[0].out_scale) / adaptive_level, 0, 1)

                return dline, dcbuf, k_out
            else:
                return blackline, np.zeros(910), np.zeros(910)

        for l in range(0, 263):
        #for l in range(60):
            curline = self.field.dspicture[self.lsl(l)].astype(np.float32)
            if l < first or l > last:
                k = np.zeros(910)
            else:
                prevline, prevcbuf, kp_out = procline(l, -1)
                nextline, nextcbuf, kn_out = procline(l, 1)

                kna = kn_out.copy()
                kna[kp_out > (3 * kn_out)] = 0

                kpa = kp_out.copy()
                kpa[kn_out > (3 * kp_out)] = 0

                k = np.min([kna, kpa], axis = 0)
                k = np.clip(kna + kpa, 0, 1)

                subset = k > 0

                sc = np.zeros_like(k)
                sc[subset] = k[subset] / (kna[subset] + kpa[subset])
                #sc = (2.0 / (kn + kp))
                #sc[sc < 1] = 1

                kprev = kpa * k * sc 
                knext = kna * k * sc

                c2d = (prevcbuf * kprev) + (nextcbuf * knext)
                #c2d /= sc

                cbuf[self.lsl(l)] = (cbuf1D[self.lsl(l)] * 1) - (c2d * k)
                #cbuf[self.lsl(l)] = cbuf1D[self.lsl(l)]

        return cbuf

    def splitIQ(self, cbuffer, line = 0):
        linephase = self.getlinephase(line)
        
        si = cbuffer[1::2].copy()
        sq = cbuffer[::2].copy()

        if not linephase:
            si[1::2] = -si[1::2]
            sq[0::2] = -sq[0::2]
        else:
            si[0::2] = -si[0::2]
            sq[1::2] = -sq[1::2]
    
        return si.astype(np.int16), sq.astype(np.int16)
    
    def calcLine19Info(self, comb_field2 = None):
        ''' returns color burst phase (ideally 147 degrees) and (unfiltered!) SNR '''
        
        # Don't need the whole line here, but start at 0 to definitely have an even #
        l19_slice = self.field.lineslice_tbc(19, 0, 40)
        l19_slice_i70 = self.field.lineslice_tbc(19, 14, 18)

        # fail out if there is obviously bad data
        if not ((np.max(self.field.output_to_ire(self.field.dspicture[l19_slice_i70])) < 100) and
                (np.min(self.field.output_to_ire(self.field.dspicture[l19_slice_i70])) > 40)):
            #logging.info("WARNING: line 19 data incorrect")
            #logging.info(np.max(self.field.output_to_ire(self.field.dspicture[l19_slice_i70])), np.min(self.field.output_to_ire(self.field.dspicture[l19_slice_i70])))
            return None, None, None

        cbuffer = self.cbuffer2D[l19_slice]
        if comb_field2 is not None:
            # fail out if there is obviously bad data
            if not ((np.max(self.field.output_to_ire(comb_field2.field.dspicture[l19_slice_i70])) < 100) and
                    (np.min(self.field.output_to_ire(comb_field2.field.dspicture[l19_slice_i70])) > 40)):
                return None, None, None

            cbuffer -= comb_field2.cbuffer[l19_slice]
            cbuffer /= 2
            
        si, sq = self.splitIQ(cbuffer, 19)

        sl = slice(110,230)
        cdata = np.sqrt((si[sl] ** 2.0) + (sq[sl] ** 2.0))

        phase = np.arctan2(np.mean(si[sl]),np.mean(sq[sl]))*180/np.pi
        if phase < 0:
            phase += 360

        # compute SNR
        signal = np.mean(cdata)
        noise = np.std(cdata)

        snr = 20 * np.log10(signal / noise)
        
        return signal / (2 * self.field.out_scale), phase, snr

    # Colorspace conversion
    def YIQtoRGB(self, Y, I, Q, irescale = True):
        R = Y + ( .956 * I) + (.621 * Q)
        G = Y - ( .272 * I) - (.647 * Q)
        B = Y - (1.106 * I) + (1.703 * Q)
        print(R.max())
        
        if irescale == False:
            #return np.clip(R, 0, 65535), np.clip(G, 0, 65535), np.clip(B, 0, 65535)
            return np.clip(R/256, 0, 65535), np.clip(G/256, 0, 65535), np.clip(B/256, 0, 65535)
        else:
            blacklevel = fields[0].hz_to_output(ldd.rf.iretohz(0))
            whitelevel = fields[0].hz_to_output(ldd.rf.iretohz(100))
            
            lrange = whitelevel - blacklevel
            R = ((R - blacklevel) / lrange) * 256
            G = ((G - blacklevel) / lrange) * 256
            B = ((B - blacklevel) / lrange) * 256
            print(R.max())
            
            return np.clip(R, 0, 255), np.clip(G, 0, 255), np.clip(B, 0, 255)

    def toYIQ(self, cbuffer = None):
        if cbuffer is None:
            cbuffer = self.cbuffer2D

        I, Q = self.splitIQ(cbuffer)
        Iout = np.repeat(I, 2)
        Qout = np.repeat(Q, 2)

        pic = self.field.dspicture.copy()

        for l in range(0, 263):
            sl_y = slice(910 * l, 910 * (l + 1))
            sl_c = slice(455 * l, 455 * (l + 1))
            lphase = 1 if self.getlinephase(l) else -1

            adj = np.zeros(910, dtype=np.float)

            adj[::4]  = (Q[sl_c][::2] * lphase)
            adj[1::4] = -(I[sl_c][::2] * lphase)
            adj[2::4] = -(Q[sl_c][1::2] * lphase)
            adj[3::4] = (I[sl_c][1::2] * lphase)
            #adj /= 1

            pic[sl_y] = np.clip(pic[sl_y].astype(float) + adj, 0, 65535)

        return (pic, Iout, Qout)

    def toRGB(self, YIQ):
        RGB = self.YIQtoRGB(*YIQ, True)
        
        rgbArray = np.zeros((263,910,3), 'uint8')
        rgbArray[...,0] = RGB[0].reshape(263,910) 
        rgbArray[...,1] = RGB[1].reshape(263,910) 
        rgbArray[...,2] = RGB[2].reshape(263,910) 

        return rgbArray    

In [ ]:

def split2D(self, first = 10, last = 262):
    adaptive_level = 30
    kfilter = [1.0] # sps.firwin(5, .1)
    #kfilter = [-.2, 1.0]
    #kfilter = sps.firwin(3, .25)

    blacklevel = fields[0].hz_to_output(ldd.rf.iretohz(0))
    blackline = np.full(910, blacklevel)

    cbuf1D = self.cbuffer1D
    cbuf = np.zeros_like(cbuf1D)

    def procline(l, gap):
        curline = self.field.dspicture[self.lsl(l)]

        if inrange((l + gap), first, last):
            dline = self.field.dspicture[self.lsl(l + gap)].astype(np.float32)
            dcbuf = self.cbuffer1D[self.lsl(l + gap)]

            k = np.fabs(curline - dline) #+ np.fabs(np.roll(curline, 1) - np.roll(nextline, 1))
            kf = sps.lfilter(kfilter, [1.0], k)
            k_out = np.clip(1 - (kf / fields[0].out_scale) / adaptive_level, 0, 1)
        
            return dline, dcbuf, k_out
        else:
            return blackline, np.zeros(910), np.zeros(910)
    
    for l in range(0, 263):
    #for l in range(60):
        curline = self.field.dspicture[self.lsl(l)].astype(np.float32)
        if l < first or l > last:
            k = np.zeros(910)
        else:
            prevline, prevcbuf, kp_out = procline(l, -1)
            nextline, nextcbuf, kn_out = procline(l, 1)

            kna = kn_out.copy()
            kna[kp_out > (3 * kn_out)] = 0

            kpa = kp_out.copy()
            kpa[kn_out > (3 * kp_out)] = 0

            k = np.min([kna, kpa], axis = 0)
            k = np.clip(kna + kpa, 0, 1)

            subset = k > 0
            
            sc = np.zeros_like(k)
            sc[subset] = k[subset] / (kna[subset] + kpa[subset])
            #sc = (2.0 / (kn + kp))
            #sc[sc < 1] = 1

            kprev = kpa * k * sc 
            knext = kna * k * sc
        
            c2d = (prevcbuf * kprev) + (nextcbuf * knext)
            #c2d /= sc

            cbuf[self.lsl(l)] = (cbuf1D[self.lsl(l)] * 1) - (c2d * k)
            #cbuf[self.lsl(l)] = cbuf1D[self.lsl(l)]

    return cbuf

c0 = CombNTSC(fields[0])

cbuf2d = split2D(c0)
YIQ = c0.toYIQ(cbuf2d)
RGB = c0.toRGB(YIQ)

#drawImage(RGB, int(910*1.5),int(526*1.5))

l = 60
#plt.plot(cbuf2d[c0.lsl(l)])
plt.plot(c0.field.dspicture[c0.lsl(l)])
plt.plot(YIQ[0][c0.lsl(l)])

In [ ]:
drawImage(RGB, int(910*1.5),int(526*1.5))

In [ ]:
l = 30
#plt.plot(c0.cbuffer1D[c0.lsl(l)])
plt.plot(kpa + kna)


In [ ]:
self = c0
first = 10
last = 262

adaptive_level = 30
kfilter = [1.0] # sps.firwin(5, .1)
#kfilter = [-.2, 1.2, -.2]
#kfilter = sps.firwin(3, .25)

blacklevel = fields[0].hz_to_output(ldd.rf.iretohz(0))
blackline = np.full(910, blacklevel)

cbuf1D = self.cbuffer1D
cbuf = np.zeros_like(cbuf1D)

def procline(l, gap):
    curline = self.field.dspicture[self.lsl(l)]

    if inrange((l + gap), first, last):
        dline = self.field.dspicture[self.lsl(l + gap)].astype(np.float32)
        dcbuf = self.cbuffer1D[self.lsl(l + gap)]

        k = np.fabs(curline - dline) #+ np.fabs(np.roll(curline, 1) - np.roll(nextline, 1))
        kf = sps.lfilter(kfilter, [1.0], k)
        k_out = np.clip(1 - (kf / fields[0].out_scale) / adaptive_level, 0, 1)

        return dline, dcbuf, k_out
    else:
        return blackline, np.zeros(910), np.zeros(910)

for l in range(61):
    curline = self.field.dspicture[self.lsl(l)].astype(np.float32)
    if l < first or l > last:
        k = np.zeros(910)
    else:
        prevline, prevcbuf, kp_out = procline(l, -1)
        nextline, nextcbuf, kn_out = procline(l, 1)

        kna = kn_out.copy()
        kna[kp_out > (3 * kn_out)] = 0

        kpa = kp_out.copy()
        kpa[kn_out > (3 * kp_out)] = 0

        k = np.min([kna, kpa], axis = 0)
        k = np.clip(kna + kpa, 0, 1)

        subset = k > 0

        sc = np.zeros_like(k)
        sc[subset] = k[subset] / (kna[subset] + kpa[subset])
        #sc = (2.0 / (kn + kp))
        #sc[sc < 1] = 1

        kprev = kpa * k * sc 
        knext = kna * k * sc

        c2d = (prevcbuf * kprev) + (nextcbuf * knext)
        #c2d /= sc

        cbuf[self.lsl(l)] = (cbuf1D[self.lsl(l)] * (1 - k)) - (c2d * k)


In [ ]:
cb = cbuf[self.lsl(l)]

In [ ]:
plt.plot(cb)

In [ ]:
plt.plot(sc)

In [ ]:
plt.plot(kprev)
plt.plot(knext)
#plt.plot(kna)

In [ ]:
plt.plot()
plt.plot(foo)

In [ ]:
l = 60
plt.plot(c0.cbuffer1D[c0.lsl(l)])
plt.plot(cbuf2d[c0.lsl(l)])

In [ ]:

RGB.mean()

In [ ]:
YIQ[2].shape

In [ ]:
c0 = CombNTSC(fields[0])
#plt.plot(c0.cbuffer1D[self.lsl(120)])
#plt.plot(c0.cbuffer2D[self.lsl(120)])
#plt.plot(cbufx[self.lsl(120)])
